In [ ]:
import  numpy as np
import pandas as pd
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from sklearn.compose import TransformedTargetRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import PredefinedSplit
import hypopt
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import QuantileTransformer, quantile_transform, StandardScaler
from sklearn.ensemble import RandomForestRegressor


from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

from sklearn.metrics import median_absolute_error, r2_score

from sklearn.model_selection import train_test_split

from matplotlib import pyplot as plt
%matplotlib inline

## Load the data

Let's load the data and have a look at it (make sure you copy the folder FFChallenge_v5 in your working directory). 

In [ ]:
# background.csv contains 4,242 rows (one per family) and 13,027 columns
#                index by challengeID: A unique numeric identifier for each child.
#                features: 13,026 background variables asked from birth to age 9, 
#                which you may use in building your model.
background = pd.read_csv('FFChallenge_v5/background.csv', sep=',', header=0,index_col=0)

# train.csv contains 2,121 rows (one per child in the training set) and 7 columns.
train = pd.read_csv('FFChallenge_v5/train.csv', sep=',', header=0, index_col=0)

#constantVariables.txt gives the column names of variables that are constant in the data.
#We recommend that the first step in any analysis be to remove the variables that are constant.
constantVariables = pd.read_csv('FFChallenge_v5/constantVariables.txt',header=None, index_col=0)

In [ ]:
# Drop constant columns
background = background.drop(constantVariables.index.values, axis = 1)

In [ ]:
# Remove all columns with more than 80% NAs
mask = (background.isna().sum(axis=0)/len(background)<0.8)
background = background[mask[mask].index]

In [ ]:
#get all negative values except -1 (refuse to answer)
negs = [-9, -8, -7, -6, -5, -4, -3, 'Other', 'Missing']

In [ ]:
#if item in list of possible negatives (other than -1 (refusal)), group them into -10
i=0
for col in background.columns:
    background[col] = [-10 if item in negs else item for item in background[col]]
    i+=1

In [ ]:
#get columns if -10 is more than 80% of values
to_remove = []
for col in background.columns:
    counts =  background[col].value_counts()
    if (counts.index[0] == -10) & (counts.iloc[0] >= 0.80 * len(background)):
        to_remove.append(col)

In [ ]:
# Drop columns where -10 is more than 80% of values
background.drop(to_remove, axis=1, inplace=True)

In [ ]:
# Get all 'object' type features
objects = [col for col in background.columns if background[col].dtype == 'object']

In [ ]:
len(objects)

In [ ]:
#convert all values in object columns to string type
for col in objects:
    background[col] = [str(item) for item in background[col]]

In [ ]:
# dummy all 'object' type features
for col in objects:
    background = pd.get_dummies(background, columns=[col], prefix = f'dummy_{col}')

In [ ]:
#Group features as categorical if less than 15 unique values and no float
cat = []
for col in background.columns:
    if col[0:5] != 'dummy':
        if (len(background[col].unique()) <= 15) & (len([item for item in background[col] if isinstance(item, float)])==0) :
            cat.append(col)

In [ ]:
len(cat)

In [ ]:
# for feature in those labelled as categorical, if most common value is not -10 and it represents 80% or more of total non-negative, values, replace column with majority or minority dummy 
to_dummy = []
for col in cat:
    if background[col].value_counts().index[0] != -10:
        if pd.DataFrame(background[col].value_counts()).iloc[0, 0] >= len(background[col]) * 0.80:
            to_dummy.append(col)
            cat.remove(col)

In [ ]:
# Create separate -10, -1, -2 dummies for all majority/minority classes defined in prior cell
for col in to_dummy:
    background[f'{col}_-10'] = (background[col] == -10)*1
    background[f'{col}_-1'] = (background[col] == -1)*1
    background[f'{col}_-2'] = (background[col] == -2)*1
    i+=1
    print(i)

In [ ]:
#create majority dummy
for col in to_dummy:
    background[f'{col}_ismax'] = (background[col] == background[col].value_counts().index[0]) * 1
    background.drop(col, axis=1, inplace=True)
    print(i)

In [ ]:
background = pd.get_dummies(background, columns=cat)

In [ ]:
#create dummies for all columns with -10, -2, -1 as values, so that can be converted to nan for future imputation
i = 0
for column in background.columns:
    if -10 in list(background[column]):
        background[f'{column}_-10'] = (background[column] == -10) * 1
    if -2 in list(background[column]): 
        background[f'{column}_-2'] = (background[column] == -2) * 1
    if -1 in list(background[column]):
        background[f'{column}_-1'] = (background[column] == -1) *1
    i+=1
    print(i)

In [ ]:
#replace all negative values with nan for future imputation
background.replace(to_replace=-10, value=np.nan, inplace=True)
background.replace(to_replace=-1, value=np.nan, inplace=True)
background.replace(to_replace=-2, value=np.nan, inplace=True)
background.replace(to_replace='nan', value=np.nan, inplace=True)

In [ ]:
#final cleaning
for column in background.columns:
    if '<' in column:
        newstr = column.replace("<", "greater")
        background = background.rename(columns={column: newstr})
    if '>' in column:
        newstr = column.replace(">", "less")
        background = background.rename(columns={column: newstr})
        
    
   

In [ ]:
#background.to_csv('cleaned_background.csv', index=True)

In [ ]:
background = pd.read_csv('cleaned_background.csv')

In [ ]:
background.set_index('challengeID', inplace=True)